In [1]:
# !git clone https://www.github.com/ivanrj7j/NLP2
# !pip install tokenizers

# Checking Data

In [5]:
from DataLoader import TrainLoader
from Tokenizer import NLPTokenizer

In [15]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np

In [37]:
from tokenizers import Tokenizer as t

In [16]:
from keras.layers import Bidirectional, LSTM, Embedding, Dense, Dropout
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping

In [24]:
trainDataset = TrainLoader(1500, 25, "/content/NLP2/data/tokenizer/tokens.json", "/content/NLP2/data/preprocessed/valid.csv",  100)
testDataset = TrainLoader(1500, 25, "/content/NLP2/data/tokenizer/tokens.json", "/content/NLP2/data/preprocessed/_test.csv",  100)

Trained using valid data to instead of whole training data

In [ ]:
for x,y in testDataset:
  x = testDataset.tokenizer.decode(x)
  y = testDataset.tokenizer.decode(y)

  data = zip(x, y)
  for _x, _y in data:
    print(_x, '\t\t', _y)
  break

Checking the testDataset loader

## Defining and training model

In [25]:
model = Sequential([
    Embedding(30003, 1000),
    Dropout(0.8),
    Bidirectional(LSTM(100)),
    Dropout(0.4),
    Dense(30003, "softmax")
    ])

In [26]:
model.compile("adam", "categorical_crossentropy", "accuracy")
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 1000)        30003000  
                                                                 
 dropout_2 (Dropout)         (None, None, 1000)        0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               880800    
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 30003)             6030603   
                                                                 
Total params: 36914403 (140.82 MB)
Trainable params: 36914403 (140.82 MB)
Non-trainable params: 0 (0.00 Byte)
__________

In [27]:
earlyStopper = EarlyStopping(patience=3)

In [28]:
history = model.fit(trainDataset.getTensorflowDataset(), epochs=50, callbacks=[earlyStopper], validation_data=testDataset.getTensorflowDataset())

Epoch 1/50
440/440 [==============================] - 128s 281ms/step - loss: 7.1969 - accuracy: 0.0846 - val_loss: 6.7304 - val_accuracy: 0.1319
Epoch 2/50
440/440 [==============================] - 113s 256ms/step - loss: 6.5328 - accuracy: 0.1421 - val_loss: 6.3712 - val_accuracy: 0.1613
Epoch 3/50
440/440 [==============================] - 117s 266ms/step - loss: 6.2308 - accuracy: 0.1660 - val_loss: 6.1411 - val_accuracy: 0.1813
Epoch 4/50
440/440 [==============================] - 115s 260ms/step - loss: 6.0144 - accuracy: 0.1800 - val_loss: 5.9892 - val_accuracy: 0.1920
Epoch 5/50
440/440 [==============================] - 115s 261ms/step - loss: 5.8523 - accuracy: 0.1891 - val_loss: 5.8882 - val_accuracy: 0.1993
Epoch 6/50
440/440 [==============================] - 114s 258ms/step - loss: 5.7224 - accuracy: 0.1960 - val_loss: 5.8161 - val_accuracy: 0.2044
Epoch 7/50
440/440 [==============================] - 114s 260ms/step - loss: 5.6144 - accuracy: 0.2015 - val_loss: 5.7508 -

In [30]:
model.save("/content/drive/MyDrive/NLP Models/bidirectional2.keras")

In [38]:
tokenizer = t.from_file("/content/NLP2/data/tokenizer/tokens.json")

In [39]:
def prepareInput(text:str) -> np.ndarray:
  tokens = np.array(tokenizer.encode(text).ids)

  if tokens.shape[0] < 25:
    tokens = np.pad(tokens, 25-tokens.shape[0], "constant")

  return tokens[-25:]

In [40]:
def predictText(start:str, limit:int=100):
  tokens = prepareInput(start)
  sentence = start
  for _ in range(limit):
    prediction = model.predict(prepareInput("Media").reshape(1, -1), verbose=False).argmax(axis=1)
    tokens = np.hstack((tokens, prediction))[-25:]
    sentence += " " + tokenizer.decode(prediction).replace('##', "")
    print(sentence, end="\r")
  print(tokens)
  return sentence

In [42]:
predictText("India is the nation")

[420 420 420 420 420 420 420 420 420 420 420 420 420 420 420 420 420 420
 420 420 420 420 420 420 420]


'India is the nation &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &# &#'

Failure! now i will try a different approach to this problem, I was avoiding to remove stop words and punctuations till now, now i am reconsidering. I will be removing stop words from tokenizer and make some other more changes!